In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!unrar x -Y "/content/drive/MyDrive/PES-Hashcode.rar" "/content/drive/MyDrive/PES-Hashcode1/"

Streaming output truncated to the last 5000 lines.
Extracting  /content/drive/MyDrive/PES-Hashcode1/PES-Hashcode/venv/Lib/site-packages/pandas/tests/arrays/categorical/test_missing.py      98%  OK 
Extracting  /content/drive/MyDrive/PES-Hashcode1/PES-Hashcode/venv/Lib/site-packages/pandas/tests/arrays/categorical/test_operators.py      98%  OK 
Extracting  /content/drive/MyDrive/PES-Hashcode1/PES-Hashcode/venv/Lib/site-packages/pandas/tests/arrays/categorical/test_replace.py      98%  OK 
Extracting  /content/drive/MyDrive/PES-Hashcode1/PES-Hashcode/venv/Lib/site-packages/pandas/tests/arrays/categorical/test_repr.py      98%  OK 
Extracting  /content/drive/MyDrive/PES-Hashcode1/PES-Hashcode/venv/Lib/site-packages/pandas/tests/arrays/categorical/test_sorting.py      98%  OK 
Extracting  /content/drive/MyDrive/PES-Hashcode1/PES-Hashcode/venv/Lib/site-packages/pandas/tests/arrays/categorical/test_subclass.py      98%  OK 
Extracting  /

In [12]:
import os, os.path
DIR = '/content/drive/MyDrive/PES-Hashcode1/PES-Hashcode/Data/pictures'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

KeyboardInterrupt: ignored

In [ ]:
for i, file in enumerate(os.listdir(DIR)):
  if i%1000==0:
    print(i)

In [13]:
drive.flush_and_unmount()

In [21]:
from collections import Counter
from itertools import chain

import numpy as np
import pandas as pd

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.utils import np_utils
from tensorflow.keras.applications import resnet50 #import ResNet50
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

#Data Loading

In [23]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(224,224), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_augmentation(self, img):
      'Apply some data augmentation'
      img = image.stateless_random_flip_left_right(img)
      img = image.stateless_random_flip_up_down(img)
      return img

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = image.load_img("/content/drive/MyDrive/PES-Hashcode/PES-Hashcode/data/images/"+str(ID)+".JPG")
            # X[i,] = np.load('data/' + ID + '.npy')

            X[i,] = self.__data_augmentation(X[i,])

            # Store class
            y[i] = self.labels[ID]

        return X, y

In [ ]:
all_images = pd.read_csv("/content/drive/MyDrive/PES-Hashcode/PES-Hashcode/data/all_images.csv")
consensus_df = pd.read_csv("/content/drive/MyDrive/PES-Hashcode/PES-Hashcode/data/consensus_data.csv")
gold_df = pd.read_csv("/content/drive/MyDrive/PES-Hashcode/PES-Hashcode/data/gold_standard.csv")



In [ ]:
class_counts = dict(Counter(chain.from_iterable(consensus_df.Species.values.tolist())))
tot = sum(chain.from_iterable(consensus_df.Species.values.tolist()))

class_freqs = {}
for k,v in class_counts.items():
  class_freqs[k] = tot/v

tot = sum(class_freqs.values())

class_weights = {}
for k,v in class_freqs.items():
  class_weights[k] = class_freqs[k]/tot


#Stage-1

In [ ]:
params = {
    "batch_size": 64,
    "n_classes": 2,
    "shuffle": True
}

X = all_images.CaptureEventID
y = [all_images.Presence]

partition = 172127-17200
X_train = X[:partition]
X_test = X[partition:]
y_train = y[:partition]
y_test = y[partition:]

X_train, X_val, y_val, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

training_generator = DataGenerator(X_train, y_train, **params)
validation_generator = DataGenerator(X_val, y_val, **params)

##Model Creation

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
from keras.engine.input_layer import Input
base_model = mobilenet_v2(weights="imagenet", include_top=False)

base_model.trainable = False

inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(2)(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, clipvalue=0.01),
              loss=keras.losses.SparseCategoricalCrossEntropy(),
              metrics = [keras.metrics.CategoricalAccuracy(), keras.metrics.TopKCategoricalAccuracy()])

In [ ]:
def scheduler(epoch, lr):
  if epoch<=18:
    return 0.01
  if epoch<=29:
    return 0.005
  if epoch<=43:
    return 0.001
  if epoch<=52:
    return 0.0005
  return 0.0001

checkpoint_filepath = '/content/drive/MyDrive/PES-Hashcode/Pes-Hashcode/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, 
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False
)

callbacks = [tf.keras.callbacks.LearningRateScheduler(scheduler), model_checkpoint_callback]

In [ ]:
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              class_weight=None,
                              epochs=55,
                              callbacks=callbacks,
                              use_multiprocessing=True,
                              workers=6)

##Visualizations

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(500)

plt.figure(figsize=(25, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
labels = ["blank", "not blank"]
predictions = model.predict(X_test)
print(classification_report(y_test, np.argmax(predictions, axis=1), target_names = labels))

In [ ]:
cm1 = confusion_matrix(y_test, np.argmax(predictions, axis=1))
df_cm = pd.DataFrame(cm1, index = [i for i in labels],
              columns = [i for i in labels])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,cmap="RdPu")
plt.savefig('confusion_mrtx3.png',bbox_inches = 'tight')

##Evaluation

In [ ]:
img_id = "ASG0000001"
images = []
img = image.load_img("/content/drive/MyDrive/PES-Hashcode/PES-Hashcode/data/images/"+str(img_id)+".JPG")
img.show()
images.append(img)
predictions = model.predict(img)
predictions = np.argmax(predictions, axis=1)
prediction = predictions[0]
if prediction == 0:
  print("Blank Image")
else:
  print("Not Blank Image")


#Stage-2

In [ ]:
labels = {}
cur_ind = 0
for val in consensus_df.Species.values():
  if val not in labels:
    labels[val] = cur_ind
    cur_ind += 1

for val in gold_df.Species.values():
  if val not in labels:
    labels[val] = cur_ind
    cur_ind += 1

